In [1]:
import fitz
import re 

def extract_sentences_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    
    for page in doc[5:]:
        text += page.get_text()
        
    #Cleaning the text
    cleaned_text = re.sub(r"[^a-zA-Z\s.]", "", text)
    # Remove extra spaces, including those before periods
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s\.', '.', cleaned_text)
    
    doc.close()
    return cleaned_text

In [2]:
sentences = extract_sentences_from_pdf("./Arjun_Thesis_Proposal.pdf")

In [3]:
print(sentences[:500])

 CHAPTER INTRODUCTION AND BACKGROUND. Biomolecular condensates Biomolecular condensates found in biology commonly function as membraneless organelles that form through liquidliquid phase separation LLPS a process where proteins and nucleic acids as semble into dynamic highlyconcentrated and spatiallysegregated dropletlike structures within cells Figure.. Figure. A cartoon showing a cell with different compartments condensates that organize biomolecules involved in similar functions. The primary 


In [12]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
tokenizer = Tokenizer()

In [6]:
# Giving number to every unique word in the data. 
tokenizer.fit_on_texts([sentences])
print(f"Total number of tokens/unique words: {len(tokenizer.word_index)}")

Total number of tokens/unique words: 1930


In [7]:
lis = list(tokenizer.word_index.items())[:10]
print(lis)

[('the', 1), ('and', 2), ('of', 3), ('to', 4), ('a', 5), ('in', 6), ('phase', 7), ('for', 8), ('we', 9), ('sequence', 10)]


In [8]:
# Break data into sentences. 
# After that make input sequence dataset. 
input_sequences = []
for sentence in sentences.split('.'):
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i + 1])

print(tokenizer.texts_to_sequences([sentence])[0])
--> 
[271, 878, 2, 879]
[36, 15, 36, 15, 407, 6, 163, 562, 79, 13, 272, 273, 21, 118, 332, 226, 7, 14, 164, 5, 165, 95, 17, 2, 563, 109, 13, 880, 26, 564, 881, 2, 882, 883, 227, 59, 408, 22]
[]
[22]
[5, 884, 409, 5, 190, 16, 145, 565, 15, 21, 410, 228, 566, 6, 274, 333]
[1, 885, 334, 275, 43, 101, 411, 335, 886, 887, 276, 91, 13, 888, 567, 412, 568, 569, 889, 2, 890, 19, 32, 891, 892, 2, 893, 894, 895, 19, 32, 17, 2, 570, 229, 336, 896, 897, 62, 44, 166, 277]
[27, 19, 278, 4, 1, 8, 898, 3, 571, 572, 45, 573, 22].............

In [9]:
# Using the zero padding to make every sequence length same. 
max_length = max([len(x) for x in input_sequences])
print(f"Max length of the sequence is {max_length}")

from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='pre')
print(padded_input_sequences)

Max length of the sequence is 53
[[   0    0    0 ...    0  271  878]
 [   0    0    0 ...  271  878    2]
 [   0    0    0 ...  878    2  879]
 ...
 [   0    0    0 ...   43 1930  129]
 [   0    0    0 ... 1930  129  330]
 [   0    0    0 ...  129  330   47]]


In [10]:
X = padded_input_sequences[ : , : -1]
Y = padded_input_sequences[ : , -1]
print(X.shape)
print(Y.shape)

(7832, 52)
(7832,)


In [11]:
# To convert output into categorical data. 
from tensorflow.keras.utils import to_categorical

Y = to_categorical(Y, num_classes= len(tokenizer.word_index) + 1)     # Added one because one hot encoding start indexing with zero and tokenizer start with one.
print(Y.shape)
print(Y[:3])

(7832, 1931)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


NN Architecture. 

First layer: Embedding layer. 
Second layer: LSTM. 
Third layer: Dense with softmax. 

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [18]:
unique_words = len(tokenizer.word_index) + 1
embedding_dim = 100
input_len = X.shape[1]
print(f'{input_len = }, {unique_words = }, {embedding_dim = }')

input_len = 52, unique_words = 1931, embedding_dim = 100


In [19]:
model = Sequential()
model.add(Embedding(unique_words, embedding_dim, input_length= input_len))
model.add(LSTM(150))                  # Total nodes on every gates.
model.add(Dense(unique_words, activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 52, 100)           193100    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 1931)              291581    
                                                                 
Total params: 635281 (2.42 MB)
Trainable params: 635281 (2.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(X, Y, epochs=100)

Epoch 1/100
245/245 [==============================] - 9s 35ms/step - loss: 6.6412 - accuracy: 0.0591
Epoch 2/100
245/245 [==============================] - 8s 33ms/step - loss: 6.1905 - accuracy: 0.0618
Epoch 3/100
245/245 [==============================] - 8s 33ms/step - loss: 6.0561 - accuracy: 0.0696
Epoch 4/100
245/245 [==============================] - 9s 36ms/step - loss: 5.8458 - accuracy: 0.0875
Epoch 5/100
245/245 [==============================] - 9s 38ms/step - loss: 5.5426 - accuracy: 0.1180
Epoch 6/100
245/245 [==============================] - 9s 37ms/step - loss: 5.1868 - accuracy: 0.1466
Epoch 7/100
245/245 [==============================] - 9s 37ms/step - loss: 4.8412 - accuracy: 0.1739
Epoch 8/100
245/245 [==============================] - 9s 38ms/step - loss: 4.5168 - accuracy: 0.2024
Epoch 9/100
245/245 [==============================] - 9s 37ms/step - loss: 4.2202 - accuracy: 0.2265
Epoch 10/100
245/245 [==============================] - 9s 37ms/step - loss: 3.945

In [28]:
# Now let's see the prediction of our model. 
import numpy as np

pred_text = ("biomolecule")

for i in range(5):               # We want to predict next ten words. 
    # tokenize the pred_text. 
    token_pred_text = tokenizer.texts_to_sequences([pred_text])[0]
    # padding
    padded_pred_text = pad_sequences([token_pred_text], maxlen=52, padding='pre')
    # prediction
    pos_next_word = np.argmax(model.predict(padded_pred_text))
    
    for word, index in tokenizer.word_index.items():
        if index == pos_next_word:
            pred_text = pred_text + " " + word
            print(pred_text)

1/1 [==============================] - 0s 11ms/step
biomolecule interactions
1/1 [==============================] - 0s 13ms/step
biomolecule interactions lead
1/1 [==============================] - 0s 13ms/step
biomolecule interactions lead to
1/1 [==============================] - 0s 11ms/step
biomolecule interactions lead to bead
1/1 [==============================] - 0s 11ms/step
biomolecule interactions lead to bead incorporation
